In [3]:
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.expressions import Beta, Variable, log, exp
from biogeme import models
from biogeme import results as res

In [5]:
#data_file = "http://transp-or.epfl.ch/data/lpmc.dat"
lpmc = pd.read_csv('lpmc10.dat', sep='\t')
lpmc

,trip_id,household_id,person_n,trip_n,travel_mode,purpose,fueltype,faretype,bus_scale,survey_year,...,dur_pt_access,dur_pt_rail,dur_pt_bus,dur_pt_int,pt_interchanges,dur_driving,cost_transit,cost_driving_fuel,cost_driving_ccharge,driving_traffic_percent
0,20,5,1,0,4,3,1,5,0.0,1,...,0.381667,0.000000,0.062222,0.000000,0,0.117222,0.00,0.41,0.0,0.097156
1,41,9,3,0,4,3,1,5,0.0,1,...,0.146944,0.000000,0.225000,0.000000,0,0.200833,0.00,0.48,0.0,0.378976
2,69,13,2,1,4,3,1,1,1.0,1,...,0.029444,0.083333,0.735833,0.398056,3,0.716944,6.00,2.16,0.0,0.582720
3,102,20,2,0,2,3,1,1,1.0,1,...,0.339722,0.183333,0.116667,0.266667,1,0.250833,3.00,0.89,0.0,0.170543
4,105,21,0,1,4,3,1,1,1.0,1,...,0.126389,0.000000,0.150000,0.000000,0,0.125833,1.50,0.37,0.0,0.154525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,80998,17591,0,5,4,3,1,5,0.0,3,...,0.189167,0.000000,0.117778,0.000000,0,0.082500,0.00,0.22,10.5,0.239057
4996,81000,17592,0,0,3,3,6,5,0.0,3,...,0.105278,0.000000,0.220278,0.000000,0,0.213611,0.00,0.52,0.0,0.412224
4997,81015,17597,0,3,4,3,1,5,0.0,3,...,0.343056,0.000000,0.177500,0.000000,0,0.189444,0.00,0.76,0.0,0.086510
4998,81041,17604,2,4,3,1,1,2,0.0,3,...,0.344444,0.316667,0.000000,0.083333,1,0.386111,1.05,0.98,0.0,0.340288


In [6]:
database = db.Database('trips', lpmc)

In [7]:
trip_id = Variable('trip_id')
household_id = Variable('household_id')
person_n = Variable('person_n')
travel_mode = Variable('travel_mode')
purpose = Variable('purpose')
fueltype = Variable('fueltype')
faretype = Variable('faretype')
bus_scale = Variable('bus_scale')
survey_year = Variable('survey_year')
travel_year = Variable('travel_year')
travel_month = Variable('travel_month')
travel_day = Variable('travel_day')
day_of_week = Variable('day_of_week')
start_time = Variable('day_of_week')
age = Variable('age')
female = Variable('female')
driving_license = Variable('driving_license')
car_ownership = Variable('car_ownership')
distance = Variable
dur_walking = Variable('dur_walking')
dur_cycling = Variable('dur_cycling')
dur_pt_access = Variable('dur_pt_access')
dur_pt_rail = Variable('dur_pt_rail')
dur_pt_bus = Variable('dur_pt_bus')
dur_pt_int = Variable('dur_pt_int')
pt_interchanges = Variable('pt_interchanges')
dur_driving = Variable('dur_driving')
cost_transit = Variable('cost_transit')
cost_driving_fuel = Variable('cost_driving_fuel')
cost_driving_ccharge = Variable('cost_driving_ccharge')
driving_traffic_percent = Variable('driving_traffic_percent')

### Model 0

In [8]:
asc_walk = Beta('asc_walk', 0, None, None, 0)
asc_cycling = Beta('asc_cycling', 0, None, None, 0)
asc_public = Beta('asc_public', 0, None, None, 0)
asc_driving = Beta('asc_driving', 0, None, None, 0)

In [9]:
cost_driving = cost_driving_fuel + cost_driving_ccharge
dur_public = dur_pt_access + dur_pt_rail + dur_pt_bus + dur_pt_int #total duration of public transportation

In [10]:
beta_cost = Beta('beta_cost', 0, None, None, 0)
beta_time = Beta('beta_time', 0, None, None, 0)

In [11]:
V_walk = asc_walk + beta_time * dur_walking
V_cycling = asc_cycling + beta_time * dur_cycling
V_driving = asc_driving + beta_time * dur_driving + beta_cost * cost_driving
V_public = asc_public + beta_time * dur_public + beta_cost * cost_transit

In [12]:
prob_walk = 1 / (1 + exp(V_cycling - V_walk) + exp(V_driving - V_walk) + exp(V_public - V_walk))
prob_cycling = 1 / (1 + exp(V_walk - V_cycling) + exp(V_driving - V_cycling) + exp(V_public - V_cycling))
prob_driving = 1 / (1 + exp(V_cycling - V_driving) + exp(V_walk - V_driving) + exp(V_public - V_driving))
prob_public = 1-prob_driving + prob_cycling + prob_walk

prob_observation = prob_walk * (travel_mode == 1) + prob_cycling * (travel_mode == 2) + prob_driving * (travel_mode == 3) + prob_public * (travel_mode == 4) 
logprob = log(prob_observation)

In [13]:
biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'model_0'

In [14]:
results = biogeme.estimate()

In [15]:
print(results.printGeneralStatistics())

Number of estimated parameters:	6
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-3511.416
Final log likelihood:	-1907.476
Likelihood ratio test for the init. model:	3207.88
Rho-square for the init. model:	0.457
Rho-square-bar for the init. model:	0.455
Akaike Information Criterion:	3826.952
Bayesian Information Criterion:	3866.056
Final gradient norm:	1.4237E-02
Nbr of threads:	16



In [16]:
results.getEstimatedParameters()

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycling,2.169646,2.383331,0.910342,0.362642,0.048636,44.610294,0.000000e+00
asc_driving,2.514422,2.383126,1.055094,0.291382,0.046437,54.147531,0.000000e+00
asc_public,-8.666554,7.148363,-1.212383,0.225366,0.023001,-376.792641,0.000000e+00
asc_walk,3.982486,2.383063,1.671163,0.094690,0.041616,95.696009,0.000000e+00
beta_cost,0.125126,0.010012,12.497377,0.000000,0.009580,13.060913,0.000000e+00
beta_time,-1.249388,0.118389,-10.553238,0.000000,0.150804,-8.284875,2.220446e-16


In [17]:
results.data.htmlFileName

'model_0.html'

### Model 1


In [18]:
asc_walk = Beta('asc_walk', 0, None, None, 0)
asc_cycling = Beta('asc_cycling', 0, None, None, 0)
asc_public = Beta('asc_public', 0, None, None, 0)
asc_driving = Beta('asc_driving', 0, None, None, 0)

In [19]:
cost_driving = cost_driving_fuel + cost_driving_ccharge
dur_public = dur_pt_access + dur_pt_rail + dur_pt_bus + dur_pt_int #total duration of public transportation

In [20]:
beta_time_walk = Beta('beta_time_walk', 0, None, None, 0)
beta_time_cycling = Beta('beta_time_cycling', 0, None, None, 0)
beta_time_driving = Beta('beta_time_driving', 0, None, None, 0)
beta_time_public = Beta('beta_time_public', 0, None, None, 0)

In [21]:
V_walk = asc_walk + beta_time_walk * dur_walking
V_cycling = asc_cycling + beta_time_cycling * dur_cycling
V_driving = asc_driving + beta_time_driving * dur_driving + beta_cost * cost_driving
V_public = asc_public + beta_time_public * dur_public + beta_cost * cost_transit

In [22]:
prob_walk = 1 / (1 + exp(V_cycling - V_walk) + exp(V_driving - V_walk) + exp(V_public - V_walk))
prob_cycling = 1 / (1 + exp(V_walk - V_cycling) + exp(V_driving - V_cycling) + exp(V_public - V_cycling))
prob_driving = 1 / (1 + exp(V_cycling - V_driving) + exp(V_walk - V_driving) + exp(V_public - V_driving))
prob_public = 1-prob_driving + prob_cycling + prob_walk

prob_observation = prob_walk * (travel_mode == 1) + prob_cycling * (travel_mode == 2) + prob_driving * (travel_mode == 3) + prob_public * (travel_mode == 4) 
logprob = log(prob_observation)

In [28]:
biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'model_1'

In [29]:
results = biogeme.estimate()

In [30]:
print(results.printGeneralStatistics())

Number of estimated parameters:	9
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-2372.019
Final log likelihood:	-1631.75
Likelihood ratio test for the init. model:	1480.538
Rho-square for the init. model:	0.312
Rho-square-bar for the init. model:	0.308
Akaike Information Criterion:	3281.501
Bayesian Information Criterion:	3340.155
Final gradient norm:	1.2934E-02
Nbr of threads:	16



In [26]:
results.getEstimatedParameters()

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycling,-2.782703,6.784647,-0.410147,6.816981e-01,0.048428,-57.460054,0.000000e+00
asc_driving,-2.870887,6.784824,-0.423134,6.721978e-01,0.104178,-27.557448,0.000000e+00
asc_public,-13.561355,20.353290,-0.666298,5.052207e-01,0.039039,-347.377694,0.000000e+00
asc_walk,-0.017513,6.784965,-0.002581,9.979406e-01,0.120145,-0.145765,8.841071e-01
beta_cost,0.062834,0.012178,5.159712,2.473297e-07,0.011852,5.301765,1.146883e-07
beta_time_cycling,3.743376,0.313148,11.954003,0.000000e+00,0.405739,9.226073,0.000000e+00
beta_time_driving,5.708331,0.458128,12.460135,0.000000e+00,0.549677,10.384891,0.000000e+00
beta_time_public,-0.003102,71.169539,-0.000044,9.999652e-01,0.368397,-0.008420,9.932818e-01
beta_time_walk,-2.117024,0.365544,-5.791437,6.978692e-09,0.563915,-3.754154,1.739278e-04


In [27]:
results.data.htmlFileName

'travel_modes~00.html'